In [1]:
import os
import ctypes
from symfit import variables, Parameter, ODEModel, D, Fit, parameters, Variable
import numpy as np
from uncertainties import ufloat as uf
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from pathlib import Path as pt
from ipywidgets import widgets
from timescan import timescanplot
from time import time as start_time

from matplotlib.widgets import Slider, CheckButtons, TextBox

#%matplotlib qt5
%matplotlib widget

In [2]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# To display the figure defined by this dict, use the low-level plotly.io.show function
import plotly.io as pio

from widgetDefinitions import createWidgets, Sliderlog
import string

alphabets = string.ascii_uppercase

def definingParameter(value):
    exec(value)
    return locals()

def getValfromAddress(address):
    return ctypes.cast(address, ctypes.py_object).value

def getValfromVariable(variable):
    address = id(variable)
    return getValfromAddress(address)

def float_slider(_value, _description, _min = -40, _max = -20, steps = 1e-3):
    return widgets.FloatLogSlider(
                value=_value,
                base=10,
                min=_min,
                max=_max,
                step=steps,
                description=_description
            )

In [3]:
style = {'description_width': 'initial', 'width':"100"}
layout=widgets.Layout(width='50%')

twidget = createWidgets(filetype="scan", multiselect=False, locationValue=r"D:\SurfDrive\THz_Measurements\CD+\timescan")
twidget.files.layout.height = "300px"

timeStartIndexWidget = widgets.BoundedIntText(value=1, description="timeStartIndex",min=0, style=style)
endtimeIndexWidget = widgets.BoundedIntText(value=-1, description="endtimeIndex", min=-1, style=style)

defaultInitialValueWidget = widgets.Checkbox(description="defaultInitialValue", value=False)
initialConditionValuesWidget = widgets.Text(description="initialConditionValues", style = style, value="", layout=layout)


tempWidget = widgets.BoundedFloatText(value=5, min=0, max=400, step=0.5, description="Temperature")
pbeforeWidget = widgets.FloatLogSlider(base=10, value=1e-8, min=-10, max=-1, description="pbefore")
paftereWidget = widgets.FloatLogSlider(base=10, value=1e-6, min=-10, max=-1, description="paftere")
srgModeWidget = widgets.Checkbox(description="SRG", value=False)

moleculeWidget = widgets.Text(description="Molecule, tag", style = style, value="CD, Ne")
tdata = widgets.BoundedFloatText( value=5, min=1, max=20, step=0.25, description='Simulation:' )
logPlot = widgets.Checkbox(description="Log", value=True)
fitWidget = widgets.Checkbox(description="fit", value=False)

oldDataToggle = widgets.Checkbox(description="old Data", value=False)

In [4]:
rate_in_text = None
rate_output = widgets.Output()

def createModal(event=None):
    
    global rate_output, rate_in_text_widget
    
    with rate_output:
        print(locals())
        
        print(rate_in_text_widget.value)
        
        exec(rate_in_text_widget.value)
        
        print(f"{locals()['ode_modal']}")
        
        return locals()["ode_modal"]
    
rate_in_text_widget = widgets.Textarea(layout=widgets.Layout(width='70%', height="300px"), style = {'description_width': 'initial'})
rate_modal_button = widgets.Button(description="Create Modal", button_style="success", layout=widgets.Layout(width='20%'), style = {'description_width': 'initial'})
rate_modal_button_clear = widgets.Button(description="Clear", button_style="danger", layout=widgets.Layout(width='20%'), style = {'description_width': 'initial'})

rate_modal_button.on_click(createModal)
rate_modal_button_clear.on_click(lambda event: rate_output.clear_output())

In [5]:
massOfReactantsWidget = widgets.Text(description="massOfReactants", style = style, value=", ", layout=layout)
nameOfReactantsWidget = widgets.Text(description="nameOfReactants", style = style, value=", ", layout=layout)
nameOfParametersForwardWidget = widgets.Text(description="nameOfParametersForward", style = style,value=", ", layout=layout)
nameOfParametersReverseWidget = widgets.Text(description="nameOfParametersReverse", style = style,value=", ", layout=layout)

ROSAA_kinetics_widgets = {"massOfReactants":massOfReactantsWidget, "nameOfReactants":nameOfReactantsWidget, 
                          "timeStartIndex":timeStartIndexWidget, "endtimeIndex":endtimeIndexWidget,
                          "initialConditionValues":initialConditionValuesWidget, "defaultInitialValue":defaultInitialValueWidget,
                          "fitODE":fitWidget, 
                        }



In [6]:
numberDensityValue = None
def numberDensity(temp, pbefore, pafter, srgMode=False):
    global numberDensityValue
    constant = 4.2e17 # 1/boltzman_constant*sqrt(T)
    C = (uf(179.26, 3.79), 1)[srgMode] # Calibration factor 
    T = uf(temp, 0.1) # Temperature
    p = pafter - pbefore
    
    numberDensityValue = (constant*C*p)/(T**0.5)
    
    print(f"{pbefore=:.2e}, {pafter=:.2e}")
    print(f"{numberDensityValue:.2e}")
    
    return numberDensityValue.nominal_value

numberDensityWidgetOutput = widgets.interactive_output(numberDensity, dict(temp=tempWidget, pbefore=pbeforeWidget, pafter=paftereWidget, srgMode=srgModeWidget))


In [7]:
time, mass = None, None
totalReactants = None

ode_modal = None
fullDataValues = None
errorDataValues = None

label = None
expTime = None

In [8]:
def ROSAA_kinetics(massOfReactants, nameOfReactants, timeStartIndex, endtimeIndex, 
                   initialConditionValues, defaultInitialValue, fitODE, filename = None):
    
    global ode_modal, fullDataValues, label, expTime, rate_in_text, rate_in_text_widget,\
            rateConstantLength, errorDataValues, totalReactants
    
    rate_in_text = []
    
    if massOfReactants == ", ": return 
    
    massLabel = massOfReactants.split(", ")
    totalReactants = len(massLabel)
    print(f"{totalReactants=}")
    
    nameOfParametersForward = ", ".join([f"k3{i}" for i in range(1, totalReactants)])
    nameOfParametersReverse = ", ".join([f"CID{i}" for i in range(1, totalReactants)])
    
    nameOfParametersForwardWidget.value = nameOfParametersForward
    nameOfParametersReverseWidget.value = nameOfParametersReverse
    
    # Getting timescan data
    expTime = time[timeStartIndex:] / 1000
    fullData = {}
    errorDataValues = {}
    
    for key in massLabel:
        fullData[key] = m[key]['y'][timeStartIndex:]
        errorDataValues[key] = m[key]['error_y']['array'][timeStartIndex:]
        
    fullDataValues = np.array(list(fullData.values()))
    errorDataValues = np.array(list(errorDataValues.values()))
    
    # Making variables
    t = Variable("t")
    rate_in_text.append(f't = Variable("t")')
    
    reactantVariable = []
    
    if nameOfReactants:
        reactantVariable = variables(nameOfReactants)
    else:
        print(f"{reactantVariable=} length not matching - {totalReactants}")
        return
    
    reactantVariable = reactantVariable[:totalReactants]
    rate_in_text.append(f'{nameOfReactants} = variables("{nameOfReactants}")')

    print(f"{reactantVariable=}")
    label = nameOfReactants.split(", ")[:totalReactants]
    
    fullDataDictionary = {key:value for key, value in zip(label, fullDataValues)}
    
    # Initial condition
    if defaultInitialValue:
        initialConditionValues = fullDataValues.T[0]
    else:
        if initialConditionValues:
            initialConditionValues = [float(_.strip()) for _ in initialConditionValues.split(",")]
        else:
            initialConditionValues = [fullDataValues[0].max()]
            for _ in range(totalReactants):
                initialConditionValues.append(0.0001)
        
        
    print(f"{initialConditionValues=}")
    initial_cond = {t:0}

    for i, variable in enumerate(reactantVariable):
        initial_cond[getValfromVariable(variable)] = initialConditionValues[i]

    print(f"{initial_cond=}")
    
    rate_in_text.append(f'initial_cond = {initial_cond}')
    
    # Rate Equation
    He = numberDensityValue.nominal_value
    
    rateConstantParametersForward = parameters(nameOfParametersForward)
    rateConstantParametersReverse = parameters(nameOfParametersReverse)
    
    rate_in_text.append(f'{nameOfParametersForward} = parameters("{nameOfParametersForward}")')
    rate_in_text.append(f'{nameOfParametersReverse} = parameters("{nameOfParametersReverse}")')
    
    if not fitODE:
        rateEquation = {"forward":[He**2 * getValfromVariable(i) for i in rateConstantParametersForward],
                        "reverse":[He * getValfromVariable(i) for i in rateConstantParametersReverse]}

        for forw, rever in zip(rateConstantParametersForward, rateConstantParametersReverse):
            getValfromVariable(forw).min = 1e-31
            getValfromVariable(forw).max = 1e-27

            getValfromVariable(rever).min = 1e-20
            getValfromVariable(rever).max = 1e-14
    
    else:
        rateEquation = {"forward":[getValfromVariable(i) for i in rateConstantParametersForward],
                        "reverse":[getValfromVariable(i) for i in rateConstantParametersReverse]}
        
        for forw, rever in zip(rateConstantParametersForward, rateConstantParametersReverse):
            getValfromVariable(forw).min = 0
            getValfromVariable(forw).max = 1

            getValfromVariable(rever).min = 0
            getValfromVariable(rever).max = 1

        
    # Formation rate
    formationRate = {}

    for i, forward, reverse in zip(range(len(rateEquation["forward"])), rateEquation["forward"], rateEquation["reverse"]):
        formationRate[f"{label[i]}"] = -getValfromVariable(forward)*getValfromVariable(reactantVariable[i]) + getValfromVariable(reverse)*getValfromVariable(reactantVariable[i+1])

    formationRateList = list(formationRate.values())
    formationRate[f"{label[-1]}"] =  -formationRateList[-1]
    formationRateList = list(formationRate.values())
    
    # Rate Modal
    rate_in_text.append('rate_model = {}')
    
    rate_model = {D(getValfromVariable(reactantVariable[0]), t): getValfromVariable(formationRateList[0])}
    rate_in_text.append(f'rate_model[D({reactantVariable[0]}, t)] = {formationRateList[0]}')
    
    for i in range(1, totalReactants-1):
        rate_model[D(getValfromVariable(reactantVariable[i]), t)] =  getValfromVariable(formationRateList[i]) -getValfromVariable(formationRateList[i-1])
        rate_in_text.append(f'rate_model[D({reactantVariable[i]}, t)] = {formationRateList[i]} - {formationRateList[i-1]}')
        
    rate_model[D(getValfromVariable(reactantVariable[-1]), t)] = getValfromVariable(formationRateList[-1])
    rate_in_text.append(f'rate_model[D({reactantVariable[-1]}, t)] = {formationRateList[-1]}')
    
    for i, j in rate_model.items():
        print(f"\n{i}:{j}")
    
    ode_modal = ODEModel(rate_model, initial=initial_cond)
    print(f"{ode_modal=}")
    
    rate_in_text.append(f'ode_modal = ODEModel(rate_model, initial=initial_cond)')
    
    rate_in_text_widget.value = "\n".join(rate_in_text)
    #display(rate_in_text_widget, rate_modal_button, rate_modal_button_clear, rate_output)
    
    assert(ode_modal.dependent_vars[0] == list(ode_modal.initial.keys())[1])
    
    if fitODE:
        
        kwargs = {"t":expTime}
        counter = 0
        for _ in nameOfReactants.split(", ")[:totalReactants]:
            kwargs[_] = fullDataValues[counter]
            counter += 1
            
        fit = Fit(ode_modal, **kwargs)
        fit_result = fit.execute()
        print(fit_result)
        
        return
    
    func()
    
ROSAA_kinetics_output = widgets.interactive_output(ROSAA_kinetics, {"filename": twidget.files, **ROSAA_kinetics_widgets})

In [9]:
rateConstantSliderWidgets = None
plot_update_output = None
k3_widgets = {}
kCID_widgets = {}
    
def make_rate_constant_sliders():
    
    global rateConstantSliderWidgets, k3_widgets, kCID_widgets
    
    k3_name = nameOfParametersForwardWidget.value.split(",")
    kCID_name = nameOfParametersReverseWidget.value.split(",")
    
    keys = simulationParameters.keys() if type(simulationParameters) == 'dict' else []
    rateConstantSliderWidgets = []
    
    for k, c in zip(k3_name, kCID_name):
        
        k = k.strip()
        c = c.strip()
        with rateOutPutWidget:
            print(f"Running")
        _k3_value = simulationParameters[k] if k in keys else 1e-30
        _k3 = widgets.FloatLogSlider(base=10, value=_k3_value, min=-40, max=-20, description=k)
        k3_widgets[k] = _k3
        
        _kCID_value = simulationParameters[c] if k in keys else 1e-15
        _kCID = widgets.FloatLogSlider(base=10, value=_kCID_value, min=-20, max=-10, description=c)
        kCID_widgets[c] = _kCID
        
        _temp =  widgets.HBox([_k3, _kCID])
        rateConstantSliderWidgets.append(_temp)
        
    rateConstantSliderWidgets = widgets.VBox(rateConstantSliderWidgets)
    plot_update_output = widgets.interactive_output(updatePlot, {**k3_widgets, **kCID_widgets})
    display(rateConstantSliderWidgets)
    
def updatePlot(**kwargs):
    
        global simulationParameters, fig, fittedPlotsCollection, fittedPlotsCollection_linear
        
        simulationTimeSet = int(expTime[-1])+1
        simulationTime = np.linspace(0, simulationTimeSet, 1000)
        
        with rateOutPutWidget:
            simulationParameters = {"t":simulationTime, **kwargs}
            simulationData = ode_modal(**simulationParameters)
            
            nameOfReactants = nameOfReactantsWidget.value.split(", ")[:totalReactants]
            for _index in range(totalReactants):
                fittedPlotsCollection[f"{nameOfReactants[_index]}"].set_ydata(simulationData[_index])
                fittedPlotsCollection_linear[f"{nameOfReactants[_index]}"].set_ydata(simulationData[_index])

            fig.canvas.draw_idle()

In [10]:
rateOutPutWidget = widgets.Output()
simulationParameters = None

textBox = None
simulationTimeSet = 5
fittedPlotsCollection = {}
fittedPlotsCollection_linear = {}
fig = None

def func():
    
    global simulationParameters, ode_modal, textBox, fittedPlotsCollection, \
        fittedPlotsCollection_linear, simulationTimeSet, fig
    
    plt.close('all')
    
    # Simulation time
    simulationTimeSet = int(expTime[-1])+1
    simulationTime = np.linspace(0, simulationTimeSet, 1000)
    
    #Modal parameters
    t = simulationTime
    _rateK3 = nameOfParametersForwardWidget.value.split(", ")
    rateK3 = {_:1e-30 for _ in _rateK3}

    _rateKCID = nameOfParametersReverseWidget.value.split(", ")
    rateKCID = {_:1e-15 for _ in _rateKCID}
    
    simulationParameters = {"t":simulationTime, **rateK3, **rateKCID}
    simulationData = ode_modal(**simulationParameters)
    
    fig, (ax, ax1) = plt.subplots(figsize=(12, 6), nrows=1, ncols=2)
    plt.subplots_adjust(left=0.1)
                             
    twidget.fig = fig
    
    nameOfReactants = nameOfReactantsWidget.value.split(", ")[:totalReactants]
    
    for _index in range(totalReactants):
        #errorbar(x, y, yerr, marker='s', mfc='red', mec='green', ms=20, mew=4)
        lg = nameOfReactants[_index] + "$^+$"
        #ax.plot(expTime, fullDataValues[_index], ".", ms=10, label=lg)
        #ax1.plot(expTime, fullDataValues[_index], ".", ms=10, label=lg)
        
        ax.errorbar(expTime, fullDataValues[_index], errorDataValues[_index], fmt=".", ms=10, label=lg)
        ax1.errorbar(expTime, fullDataValues[_index], errorDataValues[_index], fmt=".", ms=10, label=lg)
        
        fittedPlotsCollection[f"{nameOfReactants[_index]}"], = ax.plot(simulationTime, simulationData[_index], f"-C{_index}")
        fittedPlotsCollection_linear[f"{nameOfReactants[_index]}"], = ax1.plot(simulationTime, simulationData[_index], f"-C{_index}")
    
            
    tag = moleculeWidget.value.split(",")[1].strip()
    
    ax.grid()
    ax1.grid()
    
    ax.minorticks_on()
    ax1.minorticks_on()
    
    plt.suptitle(f"{twidget.files.value}: {tag} number density: {numberDensityValue.nominal_value:.2e}/cm3, Temp: {tempWidget.value}K")
    
    ax.set_yscale("log")
    ax1.legend(title="Molecules")
    
    ax.set(xlabel="Time (s)", ylabel="Counts", title="YScaling: Log")
    ax1.set(xlabel="Time (s)", title="YScaling: Linear")
    
    plt.show()
    
    make_rate_constant_sliders()
    

In [11]:
display(twidget.update_files_button, twidget.location, twidget.files, oldDataToggle)
twidget.files.options = (f"{_}".replace(".txt", ".scan") for _ in pt(twidget.location.value).glob("*_timescan.txt"))

def init(location, filename, moleculeName, oldData=False):
    
    global time, mass, m, totalReactants
    
    if not filename: return print("No files selected")

    file = pt(location) / filename
    
    if oldData:
        file = pt(location) / f"{pt(filename).stem}.txt"
        time, *massesData = np.genfromtxt(file).T

        with open(file) as f:
            for i in f.readlines():
                if i.startswith("#Time"):
                    masses = [_.strip() for _ in i.split("\t")[1:-1]]
                    mass = [float(_) for _ in masses[:-1]]
                    
        m = {}
        for _, _data in enumerate(massesData):
            m[f"{masses[_]}u"] = {"x":list(time), "y":list(_data), "name":f"{masses[_]}u", 'mode': 'lines+markers'}
        
    else:
        time, mean, error, mass, t_res, t_b0  = timescanplot(file).get_data()
        m = timescanplot(file).get_fullmass()

    data = list(m.values())
    #iplot(data)
    fig = dict({
        "data": data,
        "layout": {"title": {"text": f"Timescan: {file.name}"}, "yaxis":{"type":"log"}}
    })
    pio.show(fig)
    
    massLabel = [f"{i}u" for i in mass]
    massOfReactantsWidget.value = ", ".join(massLabel)
    
    totalReactants = len(massLabel)
    
    twidget.savefilename.value = f"{pt(filename).stem}"
    
    molecule = moleculeName.split(",")[0].strip()
    tagName = moleculeName.split(",")[1].strip()
    moleculeTag = [molecule, f"{molecule}{tagName}"]
    
    for i in range(2, totalReactants):
        _temp = f"{molecule}{tagName}{i}"
        moleculeTag.append(_temp)
    
    nameOfReactantsWidget.value = ", ".join(moleculeTag)
    
    display(tempWidget, pbeforeWidget, paftereWidget, srgModeWidget, numberDensityWidgetOutput)
    print(f"{numberDensityWidgetOutput=}")
    
    
out = widgets.interactive_output(init, dict(location=twidget.location, filename=twidget.files,
                        oldData=oldDataToggle, moleculeName=moleculeWidget)
                         )

display(out)

ROSAA_kinetics_widget_display = widgets.VBox([
    widgets.HBox([moleculeWidget]),
    widgets.HBox([massOfReactantsWidget, nameOfReactantsWidget]),
    widgets.HBox([nameOfParametersForwardWidget, nameOfParametersReverseWidget]),
    widgets.HBox([timeStartIndexWidget, endtimeIndexWidget]),
    widgets.HBox([defaultInitialValueWidget, initialConditionValuesWidget]),
    widgets.HBox([fitWidget]),
])

rateOutPutClearButtonWidget = widgets.Button(description="Clear Output", button_style="danger", layout=widgets.Layout(width='20%'), style = {'description_width': 'initial'})
rateOutPutClearButtonWidget.on_click(lambda event: rateOutPutWidget.clear_output())

display(ROSAA_kinetics_widget_display, ROSAA_kinetics_output, rateOutPutClearButtonWidget, rateOutPutWidget)
display(twidget.savedpi, twidget.savefilename, twidget.savebutton, twidget.closebutton, twidget.output)

Button(button_style='success', description='update location', layout=Layout(width='20%'), style=ButtonStyle())

Text(value='D:\\SurfDrive\\THz_Measurements\\CD+\\timescan', description='scan location', layout=Layout(width=…

Select(description='scan files', layout=Layout(height='300px', width='70%'), options=(), style=DescriptionStyl…

Checkbox(value=False, description='old Data')

Output()

...

IntText(value=140, description='savedpi', step=2)

Text(value='', description='savefilename')

Button(button_style='success', description='Save', style=ButtonStyle())

Button(button_style='danger', description='Clear', style=ButtonStyle())

Output()

In [12]:
def srg_calib():
    
    plt.close("all")
    srg_loc = pt(r"D:\SurfDrive\THz_Measurements\CD+\timescan\reports")
    srg_file = srg_loc / "srg_calibration.txt"
    print(f"File exists: {srg_file.exists()}")

    trap, srg = np.genfromtxt(srg_file).T

    # Define a model to fit to.
    def linearEq(x, slope, offset):
        return slope*x + offset

    popt, pcov = curve_fit(linearEq, trap, srg)
    perr = np.sqrt(np.diag(pcov))

    print(popt, perr)

    xfit = np.linspace(0, trap.max()+0.2*trap.max(), 50)
    yfit = linearEq(xfit, *popt)

    fig, ax = plt.subplots(dpi=160)
    ax.plot(trap, srg, ".C0")
    ax.plot(xfit, yfit, "-C0")
    plt.ticklabel_format(axis='both', style='sci', scilimits=(-6, -6))

    ax.set(title="SRG calibration for He @5K", xlabel="Trap (mBar)", ylabel="SRG (mBar)")
    ax.legend(["Measured", "C$_{He, 5K} = $"+f"{popt[0]:.2f}({perr[0]:.2f})"], title="Labbook #?; 3/9/20")

    fig.savefig(srg_loc/"srg_calibration.pdf")
    fig.savefig(srg_loc/"srg_calibration.png")

    print("Figure saved")

#srg_calib()